Imports and Setup

In [ ]:
import requests
from langchain import LangChain, LLMChain, Chain
from langchain.llms import OpenAI
from langchain.schema import Document
import os
from dotenv import load_dotenv

In [ ]:

# Define your API URL
load_dotenv()
load_dotenv()

RELIEFWEB_API_URL = os.getenv('RELIEFWEB_API_URL')

def get_rweb_data(query: dict, endpoint: str) -> list:
    url = f"{RELIEFWEB_API_URL}/{endpoint}"
    response = requests.post(url, json=query)
    if response.status_code == 200:
        answer = response.json()
    else:
        return f"No data was returned for query: {query}"
    
    results = []
    for article in answer.get("data", []):
        article_url = article["fields"]["url"]
        article_response = requests.get(article_url)
        web_content = BeautifulSoup(article_response.text, "html.parser").get_text()
        article["fields"]["body"] = web_content
        results.append(article["fields"])
    
    return results


LangChain Document Loader

In [ ]:
class ReliefWebDocumentLoader:
    def __init__(self, query: dict, endpoint: str):
        self.query = query
        self.endpoint = endpoint
    
    def load_documents(self) -> list:
        data = get_rweb_data(self.query, self.endpoint)
        documents = [Document(text=item["body"]) for item in data]
        return documents


Chain of process & responses to queries

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Prompt template
prompt_template = PromptTemplate.from_template(
    """
    The following is information extracted from humanitarian reports:
    {content}
    Please summarize the key points about {query}.
    """
)

# Define an LLMChain with OpenAI or another language model
llm_chain = LLMChain(
    llm=OpenAI(api_key='your-openai-api-key'),
    prompt_template=prompt_template
)

# LangChain chain
class ReliefWebChain(Chain):
    def __init__(self, document_loader: ReliefWebDocumentLoader, llm_chain: LLMChain):
        self.document_loader = document_loader
        self.llm_chain = llm_chain

    def run(self, query: str) -> str:
        documents = self.document_loader.load_documents()
        content = "\n".join(doc.text for doc in documents)
        result = self.llm_chain.run(content=content, query=query)
        return result


In [ ]:
if __name__ == "__main__":
    query = "sudan crisis"
    document_loader = ReliefWebDocumentLoader(
        query={"appname": "myapp", "query": {"value": query, "operator": "AND"}},
        endpoint="reports"
    )
    chain = ReliefWebChain(document_loader=document_loader, llm_chain=llm_chain)
    result = chain.run(query)
    print(result)
